# Create Data Set with All Nesesory Indicators

In [1]:
import yfinance as yf
import datetime
import numpy as np
from ta import momentum, trend
import pandas as pd

def Get_Main_Data(symbol):
    # symbol = "HDFCBANK.NS"
    # HDFCBANK.NS
    ticker = yf.Ticker(symbol)
    end_date = datetime.date.today()
    df = ticker.history(period="80d", interval="1d") # NOT CHNAGE 
    nifty_50_df = yf.download(symbol, period="50d", interval="5m") # YOU CAB CHANGE TIME FRAME 
    
    df.reset_index(inplace=True)
    df['Date'] = df['Date'].dt.date
    
    df['5EMA'] = trend.EMAIndicator(df['Close'], window=10).ema_indicator()
    
    # Create boolean conditions for uptrend and downtrend
    uptrend_condition = df['5EMA'] > df['5EMA'].shift(1)
    downtrend_condition = df['5EMA'] < df['5EMA'].shift(1)
    
    # Count consecutive True values within a rolling window
    consecutive_uptrends = uptrend_condition.rolling(window=3).sum()
    consecutive_downtrends = downtrend_condition.rolling(window=3).sum()
    
    # Use the consecutive counts to determine trend
    df['Day_Trend'] = np.where(consecutive_uptrends >= 2, 'Uptrend', np.where(consecutive_downtrends >= 2, 'Downtrend', 'Sideways'))
    
    df['Day_Candle'] = np.where(df['Close'] > df['Open'], 'Green', 'Red')
    
    for i in range(1, 3):
        df[f'{i}Prev_Day_Close'] = df['Close'].shift(i)
        df[f'{i}Prev_Day_High'] = df['High'].shift(i)
        df[f'{i}Prev_Day_Low'] = df['Low'].shift(i)
        df[f'{i}Prev_Day_Candle'] = df['Day_Candle'].shift(i)
        
    df['Day_Open'] =  df['Open'] 
    df['Day_High'] =  df['High']
    df['Day_Low'] =  df['Low']
    df['Day_Close'] =  df['Close']

    df['Market_Opening_Gap'] = df['Day_Open'] - df['1Prev_Day_Close']

    # print("MAX",max(df['Day_High']),df['1Prev_Day_High'])
    # print("MIN",min(df['Day_Low']),df['1Prev_Day_Low'])
    
    columns_to_drop = ['Volume', 'Dividends', 'Stock Splits', '5EMA', 'Low', 'Close', 'High', 'Open']
    df.drop(columns=columns_to_drop, inplace=True)
    nifty_50_df.reset_index(inplace=True)
    nifty_50_df['Date'] = nifty_50_df['Datetime'].dt.date
    nifty_50_df['Time'] = nifty_50_df['Datetime'].dt.time

    # --------------------------
    
    nifty_50_df['5EMA'] = trend.EMAIndicator(nifty_50_df['Close'], window=10).ema_indicator()
    
    # Calculate consecutive uptrends and downtrends
    uptrend_condition = nifty_50_df['5EMA'] > nifty_50_df['5EMA'].shift(1)
    downtrend_condition = nifty_50_df['5EMA'] < nifty_50_df['5EMA'].shift(1)
    
    consecutive_uptrends = uptrend_condition.rolling(window=4).sum()
    consecutive_downtrends = downtrend_condition.rolling(window=4).sum()
    
    # Determine trend based on conditions
    nifty_50_df['Trend'] = np.where((consecutive_uptrends == 4), 'Uptrend',
                                    np.where((consecutive_downtrends == 4), 'Downtrend', 'Sideways'))


    nifty_50_df['Candle'] = np.where(nifty_50_df['Close'] > nifty_50_df['Open'], 'Green', 'Red')
    nifty_50_df['RSI'] = round(momentum.RSIIndicator(nifty_50_df['Close'], window=3).rsi(),2)
    nifty_50_df['RSI_EMA'] = trend.EMAIndicator(nifty_50_df['RSI'], window=6).ema_indicator()
    nifty_50_df['RSI_check_30'] = nifty_50_df['RSI'].rolling(window=3).apply(lambda x: any(x < 30), raw=True).astype(bool)
    nifty_50_df['RSI_check_70'] = nifty_50_df['RSI'].rolling(window=3).apply(lambda x: any(x > 70), raw=True).astype(bool)
    
    nifty_50_df['10EMA'] = trend.EMAIndicator(nifty_50_df['Close'], window=10).ema_indicator()
    nifty_50_df['Prev_10EMA'] = nifty_50_df['10EMA'].shift(1)
    
    del nifty_50_df['Datetime']
    del nifty_50_df['Adj Close']
    del nifty_50_df['Volume']
    
    # Convert the 'Date' column in df to a common format
    df['Date'] = pd.to_datetime(df['Date'])
    nifty_50_df['Date'] = pd.to_datetime(nifty_50_df['Date'])
    merged_df = df.merge(nifty_50_df, on='Date', how='inner')
    desired_columns = [
            'Date',
            'Time',
            'Market_Opening_Gap',
            'Open',
            'High',
            'Low',
            'Close',
            'Candle',
            'RSI',
            'RSI_EMA',
            'RSI_check_30',
            'RSI_check_70',
            '10EMA',
            'Prev_10EMA',
            'Trend',
            'Day_Open',
            'Day_High',
            'Day_Low',
            'Day_Close',
            'Day_Trend',
            'Day_Candle',
            '1Prev_Day_Close',
            '1Prev_Day_High',
            '1Prev_Day_Low',
            '1Prev_Day_Candle',
            '2Prev_Day_Close',
            '2Prev_Day_High',
            '2Prev_Day_Low',
            '2Prev_Day_Candle'
        ]
    
    merged_df = merged_df[desired_columns]

    return round(merged_df,2)
    
data  = Get_Main_Data("^NSEI")

[*********************100%%**********************]  1 of 1 completed


# Find Pattern 

In [2]:
def find_pattern_all(df):
    df["upper_shadow"] = df['High'] - df[['Open', 'Close']].max(axis=1)
    df["lower_shadow"] = df[['Open', 'Close']].min(axis=1) - df['Low']
    df["body"] = abs(df["Close"] - df["Open"])
    df["full"] = df['High'] - df['Low']

    df['Prev_Open'] = df['Open'].shift(1)
    df['Prev_Close'] = df['Close'].shift(1)
    df['Prev_High'] = df['High'].shift(1)
    df['Prev_Low'] = df['Low'].shift(1)
    df['Prev_Candle'] = df['Candle'].shift(1)

    df['Next_Open'] = df['Open'].shift(-1)
    df['Next_Close'] = df['Close'].shift(-1)
    df['Next_High'] = df['High'].shift(-1)
    df['Next_Low'] = df['Low'].shift(-1)
    df['Next_Candle'] = df['Candle'].shift(-1)

    n = 5  # common variable for doji, dragonfly doji, gravestone doji

    df['Engulfing_Bullish'] = (df["Prev_Candle"] == "Red") & (df["Open"] <= df["Prev_Close"]) & (df["Close"] >= df["Prev_Open"]) & (df['Next_Candle'] == "Green")
    df['Engulfing_Bearish'] = (df["Prev_Candle"] == "Green") & (df["Open"] >= df["Prev_Close"]) & (df["Close"] <= df["Prev_Open"]) & (df['Next_Candle'] == "Red")
    df['Doji'] = (df["body"] < df['full'] / n) & (df['upper_shadow'] >= n * df["body"]) & (df['lower_shadow'] >= n * df["body"])
    df['Dragonfly_Doji_Bullish'] = (df["body"] < df['full'] / n) & (df['upper_shadow'] * 2 < df['lower_shadow'])
    df['Gravestone_Doji_Bearish'] = (df["body"] < df['full'] / n) & (df['upper_shadow'] > df['lower_shadow'] * 2)
    df['Evening_Bearish'] = (df["Prev_Candle"] == "Green") & (df["Next_Candle"] == "Red") & (abs(df["Prev_Close"] - df["Prev_Open"]) > df["body"])
    df['Morning_Bullish'] = (df["Prev_Candle"] == "Red") & (df["Next_Candle"] == "Green") & (abs(df["Prev_Close"] - df["Prev_Open"]) > df["body"])
    df['Hammer_Bullish'] = df['upper_shadow'] * 2 < df['lower_shadow']
    df['Shooting_Star_Bearish'] = df['upper_shadow'] > df['lower_shadow'] * 2
    df['Harami_Bearish'] = (df["Prev_Candle"] == "Green") & (abs(df["Prev_Close"] - df["Prev_Open"]) > df["body"] * 2) & (df["High"] < df["Prev_High"]) & (df["Close"] < df["Prev_Close"])
    df['Harami_Bullish'] = (df["Prev_Candle"] == "Red") & (abs(df["Prev_Close"] - df["Prev_Open"]) > df["body"] * 2) & (df["High"] < df["Prev_High"]) & (df["Close"] < df["Prev_Close"])

    bullish_columns = [col for col in df.columns if 'Bullish' in col and df[col].eq(True).any()]
    bearish_columns = [col for col in df.columns if 'Bearish' in col and df[col].eq(True).any()]

    df['Bullish_Count'] = df[bullish_columns].eq(True).sum(axis=1)
    df['Bearish_Count'] = df[bearish_columns].eq(True).sum(axis=1)

    df['Bullish_Count_Avg'] = df['Bullish_Count'].rolling(window=4).sum()
    df['Bearish_Count_Avg'] = df['Bearish_Count'].rolling(window=4).sum()


    df['Pattern_Name'] = df[bullish_columns + bearish_columns].apply(lambda x: x.index[x].tolist(), axis=1)
    
    delete_columns = ['upper_shadow', 'lower_shadow', 'body', 'full', 'Next_Open', 'Next_Close', 'Next_High', 'Next_Low', 'Next_Candle', 'Engulfing_Bullish', 'Engulfing_Bearish', 'Doji', 'Dragonfly_Doji_Bullish', 'Gravestone_Doji_Bearish', 'Evening_Bearish', 'Morning_Bullish', 'Hammer_Bullish', 'Shooting_Star_Bearish', 'Harami_Bearish', 'Harami_Bullish']
    df.drop(columns = delete_columns, inplace=True)
    return df

newdata = find_pattern_all(data)
newdata = newdata.drop(index=range(20))
newdata.reset_index(drop=True, inplace=True)
newdata.iloc[-1]

Date                      2023-11-10 00:00:00
Time                                 15:25:00
Market_Opening_Gap                     -43.45
Open                                  19439.1
High                                  19450.8
Low                                  19437.75
Close                                19443.45
Candle                                  Green
RSI                                     93.67
RSI_EMA                                 84.86
RSI_check_30                            False
RSI_check_70                             True
10EMA                                19417.28
Prev_10EMA                           19411.46
Trend                                 Uptrend
Day_Open                             19351.85
Day_High                              19451.3
Day_Low                              19329.45
Day_Close                            19425.35
Day_Trend                             Uptrend
Day_Candle                              Green
1Prev_Day_Close                   

# CE SIDE TRED

In [3]:
CE_HOLDING = False
ce_current_position = 0
ce_inisal_amount = 0
CE_data = pd.DataFrame(columns=['Entry Position', 'Entry DateTime', 'Exit DateTime', 'Current P&L', 'Exit Position','Symbol'])
from datetime import datetime

In [4]:
# # lOGIC 1 
# def CE_ENTRY(row):
#     ema_check1 = row['Candle'] == "Green" and row['Prev_High'] < row['Prev_10EMA'] and row['Prev_High'] < row['Close'] and row['High'] >= row['10EMA'] >= row['Low'] # EMA Crossover
#     rsi_check = (row['RSI_check_30'] and row['RSI_EMA']<= 60 )
#     trend_check = row['Trend'] != "Sideways" and row['Day_Trend'] != "Downtrend"
    
#     if ema_check1 and rsi_check and trend_check:
#         # print("ENTRY",row['Date'],row['Time'])
#         return True
#     else:
#         False

# def CE_EXIT(row):
#     ema_check1 = row['Prev_Low'] > row['Low'] and  row['Prev_Open'] > row['Close'] and row['Low'] <= row['10EMA'] and  row['Candle'] == "Red"  # EMA CROSS OVER
#     rsi_check = row['RSI_check_70'] and row['RSI_EMA'] >= 60
    
#     if ema_check1 and rsi_check :
#         # print("EXIT",row['Date'],row['Time'])
#         return True
#     else:
#         return False


In [5]:
# LOGIC 2

def CE_ENTRY(row):
    rsi_check = (row['RSI_EMA'] <= 30 or row['RSI_check_30'] )
    if rsi_check:
        # print("ENTRY",row['Date'],row['Time'])
        return True
    else:
        False

def CE_EXIT(row):
    rsi_check = (row['RSI_EMA'] >= 70 or row['RSI_check_70'] )
    if rsi_check :
        # print("EXIT",row['Date'],row['Time'])
        return True
    else:
        return False


In [6]:

def find_10EMA_CE(row):
    global CE_HOLDING
    global ce_current_position
    global ce_inisal_amount
    global CE_data
    if CE_ENTRY(row):
        if not CE_HOLDING:
            ce_current_position = row['Close']
            CE_HOLDING = True
            entry_data = {
                'Entry Position': row['Close'],
                'Entry DateTime': datetime.combine(row['Date'], row['Time']),
                'Exit DateTime': None,
                'Current P&L': None,
                'Exit Position': None,
                'Symbol': "CE"
            }
            CE_data = pd.concat([CE_data, pd.DataFrame([entry_data])], ignore_index=True)
            return
    if CE_EXIT(row):
        if CE_HOLDING:
            current_pnl = (row['Close'] - ce_current_position)
            ce_inisal_amount += current_pnl
            CE_HOLDING = False
            CE_data.loc[CE_data.index[-1], 'Exit DateTime'] = datetime.combine(row['Date'], row['Time'])
            CE_data.loc[CE_data.index[-1], 'Current P&L'] = current_pnl
            CE_data.loc[CE_data.index[-1], 'Exit Position'] = row['Close']
            return
    
newdata.apply(find_10EMA_CE, axis=1)
if CE_HOLDING:
    print("HOLDING")
    last_price_today = round(newdata.iloc[-1]['Close'],2)
    current_pnl  = last_price_today-ce_current_position
    ce_inisal_amount = ce_inisal_amount + current_pnl
    CE_data.loc[CE_data.index[-1], 'Exit DateTime'] =  f"{newdata.iloc[-1]['Date']} {newdata.iloc[-1]['Time']}"
    CE_data.loc[CE_data.index[-1], 'Current P&L'] = current_pnl
    CE_data.loc[CE_data.index[-1], 'Exit Position'] = last_price_today
    
print("TOTAL CE: ",ce_inisal_amount)
print("TOATL TRED : ",CE_data.shape[0])

TOTAL CE:  -34.0500000000211
TOATL TRED :  224


In [7]:
CE_data['Entry DateTime'] = pd.to_datetime(CE_data['Entry DateTime'])
CE_data['Exit DateTime'] = pd.to_datetime(CE_data['Exit DateTime'])
CE_data['Tred Time'] = (CE_data['Exit DateTime'] - CE_data['Entry DateTime']).dt.total_seconds() / 60
CE_data

,Entry Position,Entry DateTime,Exit DateTime,Current P&L,Exit Position,Symbol,Tred Time
0,19443.15,2023-08-30 11:40:00,2023-08-30 11:55:00,3.8,19446.95,CE,15.0
1,19442.30,2023-08-30 12:20:00,2023-08-30 13:10:00,1.25,19443.55,CE,50.0
2,19432.65,2023-08-30 13:40:00,2023-08-31 09:15:00,-56.6,19376.05,CE,1175.0
3,19319.85,2023-08-31 09:25:00,2023-08-31 09:45:00,59.85,19379.7,CE,20.0
4,19345.00,2023-08-31 10:05:00,2023-08-31 11:20:00,-20.25,19324.75,CE,75.0
...,...,...,...,...,...,...,...
219,19352.95,2023-11-10 09:25:00,2023-11-10 10:15:00,4.85,19357.8,CE,50.0
220,19359.50,2023-11-10 11:00:00,2023-11-10 11:20:00,10.2,19369.7,CE,20.0
221,19373.95,2023-11-10 11:25:00,2023-11-10 11:30:00,-1.8,19372.15,CE,5.0
222,19391.50,2023-11-10 13:10:00,2023-11-10 13:35:00,19.35,19410.85,CE,25.0


In [8]:
newdata.columns

Index(['Date', 'Time', 'Market_Opening_Gap', 'Open', 'High', 'Low', 'Close',
       'Candle', 'RSI', 'RSI_EMA', 'RSI_check_30', 'RSI_check_70', '10EMA',
       'Prev_10EMA', 'Trend', 'Day_Open', 'Day_High', 'Day_Low', 'Day_Close',
       'Day_Trend', 'Day_Candle', '1Prev_Day_Close', '1Prev_Day_High',
       '1Prev_Day_Low', '1Prev_Day_Candle', '2Prev_Day_Close',
       '2Prev_Day_High', '2Prev_Day_Low', '2Prev_Day_Candle', 'Prev_Open',
       'Prev_Close', 'Prev_High', 'Prev_Low', 'Prev_Candle', 'Bullish_Count',
       'Bearish_Count', 'Bullish_Count_Avg', 'Bearish_Count_Avg',
       'Pattern_Name'],
      dtype='object')